# `setigen` crash course for BLUSE@MeerKAT

`setigen` supports many ways of creating synthetic data.  This notebook showcases a small but practical subset of these techniques to emulate the data processed by the BLUSE Doppler drift search algorithm.

## Simulating BLUSE data analysis

The BLUSE commensal backend upchannelizes, beamforms, and doppler drift searches the data in situ using a unified software package called `seticore`.  We will be emulating some of the stages of `seticore` processing using `setigen` to generate simulated beamformer output and `turboSETI` to search for Doppler drifting signals.  This is a fairly close approximation to what the unified `seticore` package does, but `turboSETI` has been designed for *single pixel* (i.e. single dish, single beam) data so it does not support some of the more advanced RFI rejection techniques, such as *multibeam excision*.  As a workaround, we will fall back to using a sequence of ON/OFF observations, known as a *cadence*, with the `find_event_pipeline` and `plot_event_pipeline` functions that we've seen in other notebooks.

#### Food for thought 🤔

How are single beam and multibeam observations similar?  How do they differ?  Why do we **not** use a cadence of observations when observing commensally?

## Channelization details

### MeerKAT channelization details

The MeerKAT F engines channelize the antenna voltages into 1024, 4096, or 32768 frequency channels when running in "1K", "4K", and "32K" modes, resp.  A *zoom mode* also exists, but BLUSE does not utilize that because it offers less bandwidth (albeit at a higher resolution).  The MeerKAT L band receiver observes sky freqencies from 856 MHz to 1712 MHz (856 MHz bandwidth).  The MeerKAT UHF receiver observes sky frequencies from 544 MHz to 1088 MHz (544 MHz bandwidth).  

### BLUSE channelization details

The BLUSE backend will further channelize, aka *up-channelize*, the F engines' channelized antenna voltages.  Due to memory constraints, only a portion of the bandwidth is processed at one time.  The BLUSE backend loops over the input channels, always processing the same total bandwidth at each iteration.  The amount processed is 1 coarse channel in 1K mode, 4 coarse channels in 4K mode, and 32 coarse channels in 32K mode.  To keep the rest of the back end processing independent of the F engine mode, the up-channeliztion process always outputs the same number of fine channels, regardless of input mode.  The channelization details and resultant fine channel spectral resolutions are summarized in the following table.

|Recevier | BW (MHz) | F engine mode |`N_b`| `N_up` |  `N_f` | Spectral resolution (Hz) |
|---------|----------|---------------|-----|--------|--------|--------------------------|
| L band  |    856   |    1K mode    |  1  | 2\**19 | 2\**29 | ~1.59 Hz                 |
| L band  |    856   |    4K mode    |  4  | 2\**17 | 2\**29 | ~1.59 Hz                 |
| L band  |    856   |   32K mode    | 32  | 2\**14 | 2\**29 | ~1.59 Hz                 |
|   UHF   |    544   |    1K mode    |  1  | 2\**19 | 2\**29 | ~1.01 Hz                 |
|   UHF   |    544   |    4K mode    |  4  | 2\**17 | 2\**29 | ~1.01 Hz                 |
|   UHF   |    544   |   32K mode    | 32  | 2\**14 | 2\**29 | ~1.01 Hz                 |

Where:

- `N_b` is the number of coarse channels processed at one time (`b` for *batch size*)
- `N_up` is the number of fine channels per coarse channel (aka the *up-channelization factor*)
- `N_f` is the total number of fine channels (i.e. `total_F_engine_channels * N_b`)


To keep things simple (less complicated), this notebook will focus only on L band 4K mode observations.  You are encouraged to branch out and try other modes or receivers once you get more comfortable with this.

To keep data sizes manageable (less unmanageable), this notebook will only simulate `N_b` coarse channels at a time.

## Let's get started!

Now that we know some of the specifics, we can get started with `setigen`.

First, the obligatory `import` statements.

In [ ]:
from astropy import units as u
import setigen as stg
import matplotlib.pyplot as plt
import os
%matplotlib inline

Here we set some variables based on the L band 4K mode entry in the table above.  Setting these in one place will make it easier for you to experiment with different settings when you're ready.

In [ ]:
BW = 856
Nc = 4096 # Total number of coarse channels (aka F engine mode)
Nb = 4
Nup = 2**17

Because we are now experts on the MeerKAT/BLUSE backend, we will use the `from_backend_params` to generate a *frame* to hold our data, but first let's define a few other variables that correspond to the keyword arguments.  Refer to the `from_backend_params` [documentation](https://setigen.readthedocs.io/en/main/setigen.html#setigen.frame.Frame.from_backend_params) for more details.

In [ ]:
# Our total number of frequency channels will be
# `Nb coarse channels * Nup up-channelization`
fchans = Nb * Nup

# obs_length is in seconds, 300 (i.e. 5 minutes) is typical
obs_length = 300

# Full-band sample rate in Hz is 2e6*BW, but for our Nb channels
# the sample rate is 2e6*BW*Nb/Nc
sample_rate = 2e6*BW*Nb/Nc

# Number of PFB channels (ie coarse channels) times 2
num_branches = 2 * Nb

# Up-channelization factor
fftlength = Nup

# Integration factor, i.e. number of spectra to integrate together (8 is typical for BLUSE)
int_factor = 8

# Freqeuncy of first channel.  Let's start at center of band
fch1 = 1.5 * BW * u.MHz

# MeerKAT frequencies are always ascending (at least so far...)
ascending = True

# Finally we can create the frame!
frame = stg.Frame.from_backend_params(fchans=fchans,
                                      obs_length=obs_length,
                                      sample_rate=sample_rate,
                                      num_branches=num_branches,
                                      fftlength=fftlength,
                                      int_factor=int_factor,
                                      fch1=fch1,
                                      ascending=ascending)

Let's see what's in our new frame.

In [ ]:
fig = plt.figure(figsize=(10, 6))
frame.plot()

The new frame is empty (i.e. full of zeros).  We need to add some background noise and some signals to it, but first note that the parameters we've chosen result in our frame having 2\**15 frequency channels and 59 time samples.

## Add noise

Adding noise is done with the `add_noise` function.  You can [read the docs](https://setigen.readthedocs.io/en/main/setigen.html?highlight=add_noise#setigen.frame.Frame.add_noise) for more details, but the basic usage shown here is pretty straight forward.  After adding noise, we plot the frame again.

In [ ]:
frame.add_noise(x_mean=10, noise_type='chi2')
fig = plt.figure(figsize=(10, 6))
frame.plot()

#### Food for thought 🤔

What happens to the frame if you change the `integration_factor`?  What happens to the noise statistics?

## Add signal

Adding a signal is done with the `add_signal` function.  You can [read the docs](https://setigen.readthedocs.io/en/main/setigen.html?highlight=add_signal#setigen.frame.Frame.add_signal) for more details.  Many types of signals have convenience wrappers around `add_signal` to make it easier to add signals of these types.  Here we use `add_constant_signal` to add a Doppler drifting signal starting at channel 200 with a drift rate of 2 Hz/s, a width of 40 Hz, and a frequency profile that is a Gaussian.  Of course, we plot the frame after to see the results.

In [ ]:
# Feel free to change these parameters
start_chan = 123456
drift_rate = 2 * u.Hz/u.s
snr = 30
width = 4 * u.Hz

frame.add_constant_signal(f_start=frame.get_frequency(start_chan),
                          drift_rate=drift_rate,
                          level=frame.get_intensity(snr=snr),
                          width=width,
                          f_profile_type='gaussian')

fig = plt.figure(figsize=(10, 6))
frame.plot()

If you used the suggested parameters, you probably won't see the signal because if the limited resolution of your display compared to the frame.  Don't worry, the signal is still in there.

## Save to file

Now that we have created our frame and added noise and a few sneaky signals to it, we can save it to a file so that someone else can try to find our signals using `turboSETI`.  (You can ignore any `WARNING` messages that may be printed.)

In [ ]:
username = os.getenv("LOGNAME")
outdir = os.path.join('/scratch/bluse', username, 'sg1')
os.makedirs(outdir, exist_ok=True)
frame.save_hdf5(filename=os.path.join(outdir, f'{username}.h5'))

## Next steps

Feel free to experiment with adding more signals with different parameters and even more types of signals.  The `setigen` docs describe [how to use prepackaged signal functions](https://setigen.readthedocs.io/en/main/basics.html#using-prepackaged-signal-functions) to add many different types of signals.  Feel free to add many signals to your frame.

If you want to start over, you can call `frame.zero_data()` to reset the frame's data to all zeros.  Don't forget to add noise again after resetting the frame's data to all zeros!  Some `add_signal` variants need noise in the frame so they can adjust their levels to match your SNR specification.

Have fun!